In [141]:
import pandas as pd

import torch.nn as nn 
from torch.nn.init import xavier_uniform_, kaiming_uniform_
import torch

from preprocessing import preprocess 

import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.init import kaiming_uniform_, xavier_uniform_
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import pandas as pd

In [142]:
df_train = pd.read_csv('datasets/train_radiomics_hipocamp.csv')
df_test = pd.read_csv('datasets/test_radiomics_hipocamp.csv')



In [158]:
class MLP_1(nn.Module):
    def __init__(self, n_inputs, n_classes):
        super(MLP_1, self).__init__()
        self.hidden1 = nn.Linear(n_inputs, 512)
        self.act1 = nn.ReLU()

        self.hidden1_1 = nn.Linear(512, 512)
        self.act1_1 = nn.ReLU()
        
        self.hidden2 = nn.Linear(512, 256)
        self.act2 = nn.ReLU()
        
        self.hidden3 = nn.Linear(256, 128)
        self.act3 = nn.ReLU()
        
        self.output = nn.Linear(128, n_classes)
        self.act4 = nn.Softmax(dim=1)

    def forward(self, X):
        X = self.hidden1(X)
        X = self.act1(X)
        X = self.hidden1_1(X)
        X = self.act1_1(X)
        X = self.hidden2(X)
        X = self.act2(X)
        X = self.hidden3(X)
        X = self.act3(X)
        X = self.output(X)
        X = self.act4(X)
        print(X)
        return X

In [154]:
from sklearn.metrics import f1_score


input
class RadiomicsClassifier:
    def __init__(self, input_size, num_classes, learning_rate=0.001, batch_size=32, epochs=25):
        self.input_size = input_size
        self.num_classes = num_classes
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.epochs = epochs

        # Define the model
        self.model = MLP_1(input_size, num_classes)

        # Loss function and optimizer
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=learning_rate)

    def preprocess_data(self, df):
        # Train-test split
        X_train, X_test, y_train, y_test, self.le = preprocess(df)
        
        # Convert to PyTorch tensors
        X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
        X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
        y_train_tensor = torch.tensor(y_train, dtype=torch.long)
        y_test_tensor = torch.tensor(y_test, dtype=torch.long)

        # Create DataLoaders
        self.train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=self.batch_size, shuffle=True)
        self.test_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor), batch_size=self.batch_size, shuffle=False)

    def train(self):
        self.model.train()
        for epoch in range(self.epochs):
            epoch_loss = 0.0
            for X_batch, y_batch in self.train_loader:
                self.optimizer.zero_grad()
                outputs = self.model(X_batch)
                loss = self.criterion(outputs, y_batch)
                loss.backward()
                self.optimizer.step()
                epoch_loss += loss.item()
            print(f"Epoch [{epoch+1}/{self.epochs}], Loss: {epoch_loss/len(self.train_loader):.4f}")

    def evaluate(self):
        self.model.eval()
        all_preds = []
        all_labels = []
        with torch.no_grad():
            for X_batch, y_batch in self.test_loader:
                outputs = self.model(X_batch)
                _, preds = torch.max(outputs, 1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(y_batch.cpu().numpy())
        accuracy = accuracy_score(all_labels, all_preds)
        f1 = f1_score(all_labels, all_preds,  average='weighted')
        print(f"Test Accuracy: {accuracy:.4f}, f1: {f1:.4f}")

    def save_model(self, filepath):
        torch.save(self.model.state_dict(), filepath)
        print(f"Model saved to {filepath}")

    def load_model(self, filepath):
        self.model.load_state_dict(torch.load(filepath))
        self.model.eval()
        print(f"Model loaded from {filepath}")

    # def predict(self, test_loader=None):
    #     if test_loader is None:
    #         test_loader = self.test_loader
    #     self.model.eval()
    #     predictions = []
    #     with torch.no_grad():
    #         for X_batch, _ in test_loader:
    #             outputs = self.model(X_batch)
    #             _, preds = torch.max(outputs, 1)
    #             predictions.extend(preds.cpu().numpy())
    #     return predictions

    def predict(self, df):
        # Preprocess the new DataFrame
        X = preprocess(df,mode="test")
        X_new = X.values  # Assuming df contains only features
        # Convert to PyTorch tensor
        X_new_tensor = torch.tensor(X_new, dtype=torch.float32)
        new_loader = DataLoader(TensorDataset(X_new_tensor), batch_size=self.batch_size, shuffle=False)

        # Make predictions
        self.model.eval()
        predictions = []
        with torch.no_grad():
            for X_batch in new_loader:
                outputs = self.model(X_batch[0])
                _, preds = torch.max(outputs, 1)
                predictions.extend(preds.cpu().numpy())
        return predictions


In [159]:
# Initialize and run the classifier
input_size = 2013  # Number of features
num_classes = 5  # Number of output classes
classifier = RadiomicsClassifier(input_size, num_classes)
classifier.preprocess_data(df_train)
classifier.train()
classifier.evaluate()


tensor([[0.2077, 0.2007, 0.1986, 0.1998, 0.1932],
        [0.2064, 0.1990, 0.1973, 0.2044, 0.1929],
        [0.2077, 0.2019, 0.2007, 0.1988, 0.1909],
        [0.2043, 0.2018, 0.1990, 0.2028, 0.1920],
        [0.2046, 0.2028, 0.1989, 0.2013, 0.1925],
        [0.2015, 0.2047, 0.1988, 0.2041, 0.1909],
        [0.2066, 0.2004, 0.1990, 0.2024, 0.1916],
        [0.2044, 0.2011, 0.2018, 0.2013, 0.1914],
        [0.2029, 0.2008, 0.2002, 0.2050, 0.1911],
        [0.2009, 0.2010, 0.2040, 0.2025, 0.1917],
        [0.2014, 0.2008, 0.2026, 0.2021, 0.1931],
        [0.2067, 0.1979, 0.2002, 0.2018, 0.1933],
        [0.2034, 0.2014, 0.2007, 0.2011, 0.1935],
        [0.2052, 0.2002, 0.2011, 0.2017, 0.1919],
        [0.2053, 0.1994, 0.1985, 0.2047, 0.1922],
        [0.2071, 0.1995, 0.1995, 0.2021, 0.1918],
        [0.2061, 0.2025, 0.1988, 0.2018, 0.1908],
        [0.2032, 0.1986, 0.2014, 0.2040, 0.1928],
        [0.2075, 0.1993, 0.1994, 0.2037, 0.1901],
        [0.2054, 0.1995, 0.1998, 0.2018, 0.1934],


In [137]:
classifier.save_model("MLP2")
# classifier.preprocess_data(df_test)  # Replace df_test with your test dataset
# X_tensor = torch.tensor(df_test, dtype=torch.float32)

Model saved to MLP2


In [138]:
# classifier.load_model("MLP2")
preds = classifier.predict(df_test)

dt_predictions = classifier.le.inverse_transform(preds)


In [139]:
submission = pd.DataFrame(
    {'RowId': df_test.index + 1, 'Result': dt_predictions})
submission

,RowId,Result
0,1,AD-AD
1,2,CN-CN
2,3,MCI-MCI
3,4,CN-CN
4,5,CN-CN
...,...,...
95,96,AD-AD
96,97,AD-AD
97,98,CN-CN
98,99,CN-CN


In [140]:
with open(f'results/mlp2.csv', 'w') as file:
    submission.to_csv(file, index=False, sep=",")